This code is based on Python 3.5.

I read files with chunks because the process is ressource demanding, especially the join operations. I use a PC with Windows 10 and 32 GB RAM, so the proposed chuck size is rather large.

A special attention is devoted to data format (for airport origin, airport destination and date) because a bad data format yields mismatch problems.


In [1]:
# Common parameters of the code
import pandas as pd 

# Important filenames
strSearchesFilename='searches.csv' 
strBookingsFilename='bookings.csv' 
strNewSearchesFilename='searcheswithbookings.csv' # output file

# To obtain a reasonable computation time, the chunk size should be large 
intChunksize=10000000 

# Columns for joining the bookings and the searches
# It is assumed that the data in searches.csv should match with the date in bookings
listColumnsSearchesInner=['Origin', 'Destination', 'Date']
listColumnsBookingInner=['dep_port', 'arr_port','cre_date           ']
listColBookingName='Booking' # new column in the searches file

The next cell defines two functions to format correctly the important columns reading from the csv files.
The problem of format causes mismatch inconsistencies when associating bookings and searches.

In [2]:
# the "strip" function removes the whitespace to avoid mismatch problem when joining the data
# The text format of 'dep_port' and 'arr_port' are not identical to 'Origin' and 'Destination'
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text
    
# I read the file and declare NaT the dates uncorrectly formatted
# The correct format is '%Y-%m-%d'
parse = lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')

I read the file and declare NaT the dates uncorrectly formatted.

The correct format is '%Y-%m-%d'

In [3]:
# retrieve the columns of "searches.csv"  (0 line is read)
dfSearchCol = pd.read_csv(strSearchesFilename,sep='^',nrows=0)
listColumnsSearches=list(dfSearchCol.columns.values)  
listAllColumns=listColumnsSearches+listColumnsBookingInner

dfSearchCol[listColBookingName]=0 # Add the new column 'booking'
usedColumns=listColumnsSearches+listColumnsBookingInner
# Create an empty CSV file only with the column names
dfSearchCol.to_csv(strNewSearchesFilename,sep='^',mode='w',index=False,header=True)  

del dfSearchCol  # delete the dataframe to save memory

The next cell works as follows:

1- I read the searches file

2- I read the bookings file

3- I use an inner joint to match the searches with the bookings (keeping only the relevant columns airport origin, airport destination and date)

4- I accumulate the records w.r.t. the bookings chunks.

5- I drop useless columns after the accumulation and the duplicated records.

6- Finally, I create a new column called 'Booking' which contains 1 in case of booking matching, and 0 otherwise. The output file is generated with a left outer joint between the searches and the inner jointed bookings.

7- The output file is writing in CSV format iteratively chunk after chunck. I delete temporary dataframes to save memory.

The output file is written in CSV format iteratively chunk after chunck. 


In [4]:
# Read searches.csv, all the columns of interests
# All the formating operations are made during the reading step in order to minimize the computational effort
# I precise the dtype of certain columns to avoid some warning: some columns are not type consistent 
# For example, a column can contain both a string and an integer.
dfSearch = pd.read_csv(strSearchesFilename,sep='^', 
                       usecols=listColumnsSearches,
                       parse_dates=[listColumnsSearchesInner[2]],date_parser=parse,
                       converters = {listColumnsSearchesInner[0] : strip,
                                     listColumnsSearchesInner[1] : strip},
                       dtype={listColumnsSearches[40]:'object',
                              listColumnsSearches[41]:'object',
                              listColumnsSearches[42]:'object',
                              listColumnsSearches[44]:'object'},
                       chunksize=intChunksize)
# Loop over the searches with chunks
for chunkS in dfSearch:
    dfConcatenated = pd.DataFrame(columns=listAllColumns) # initialization: empty dataframe whose columns correspond
                                                       # to the columns of the final output file
    # read bookings.csv, only the column of interests given in listColumnsBookingInner and chunk per chunk 
    # in order to retrieve a corresponding record within chunkS
    # All the formating operations are made during the reading step in order to minimize the computational effort
    dfBook = pd.read_csv(strBookingsFilename,sep='^', 
                         usecols=listColumnsBookingInner,
                         parse_dates=[listColumnsBookingInner[2]],date_parser=parse,
                         converters = {listColumnsBookingInner[0] : strip, # remove whitespaces in the columns
                                       listColumnsBookingInner[1] : strip},
                         chunksize=intChunksize) # read the CSV file, keeping only columns in listColumnsBookingInner
 
    # Loop over the dataframe dfBook for the bookings
    for chunkB in dfBook:
        # The inner joint keeps only the common records
        dfMergeSB = pd.merge(chunkS, chunkB, 
                             left_on=listColumnsSearchesInner, 
                             right_on=listColumnsBookingInner, 
                             how='inner')       
        dfConcatenated=pd.concat([dfConcatenated, dfMergeSB]) # I append the new relevant records to the previous ones
      
    # I process below the dataframe dfConcatenated
    # After reading all the dfBook, I drop the possible duplicates (w.r.t. colBook) inside the global dafa frame     
    dfConcatenated.drop(listColumnsBookingInner, axis=1,inplace=True)
    #dfConcatenated contains the rows from chunkS which are associated to at least one booking    
    dfConcatenated=dfConcatenated.drop_duplicates().reset_index(drop=True)  # remove the possible duplicates of a row
    dfConcatenated[listColBookingName]=1 # create a column of 1 with the name defined in listColBookingName
    # If dfConcatenated is empty, the new column 'booking' will contain NaN values after the following merge
    dfMergeSContatenated = pd.merge(chunkS, dfConcatenated, 
                                    left_on=listColumnsSearches, 
                                    right_on=listColumnsSearches, 
                                    how='left')     
    # Transform NaN into 0 and 1 into 1    
    dfMergeSContatenated[listColBookingName]=-(pd.isnull(dfMergeSContatenated[listColBookingName]).astype(int)-1)
    # Append the resulting data frame to the file named in strNewSearchesFilename
    dfMergeSContatenated.to_csv(strNewSearchesFilename,sep='^',mode='a',index=False,header=False)
    
    del dfMergeSContatenated  # delete a dataframe to save memory
    del dfConcatenated        # delete a dataframe to save memory    
    del dfMergeSB             # delete a dataframe to save memory


As a micro-bonus, I compute the percentage of bookings w.r.t. searches.

The reading of the new file does not use chuncks to facilitate and accelerate the process.

In [5]:
# retrieve the column 'Booking'
dfSearchesBooking = pd.read_csv(strNewSearchesFilename,sep='^',usecols=[listColBookingName])
# compute the percentage
floatBookingMatchingPercentage = sum(dfSearchesBooking[listColBookingName])/dfSearchesBooking.shape[0]*100
print("The booking matching percentage is ",floatBookingMatchingPercentage.round(2))

The booking matching percentage is  3.56
